In [ ]:
import pandas as pd
import pickle
import numpy as np

In [ ]:
predictions_over_time_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/with_feature_aggregration/testing/predictions_over_timesteps.pkl'
features_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/preprocessed_features_01012023_233050.csv'
labels_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/preprocessed_outcomes_01012023_233050.csv'

In [ ]:
outcome = '3M mRS 0-2'
test_size = 0.2
seed = 42
n_time_steps = 72
n_splits = 5

In [ ]:
with open(predictions_over_time_path, 'rb') as handle:
    predictions_over_time = pickle.load(handle)

In [ ]:
from prediction.outcome_prediction.data_loading.data_loader import load_data

pids, training_data, test_data, splits, test_features_lookup_table = load_data(features_path, labels_path, outcome,
                                                                               test_size, n_splits, seed)


In [ ]:
test_X_np, test_y_np = test_data

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_scores = []
for ts in range(n_time_steps):
    modified_time_steps = ts + 1
    y_pred = predictions_over_time[ts]
    roc_auc_scores.append([modified_time_steps, roc_auc_score(test_y_np, y_pred)])

In [ ]:
results_df = pd.DataFrame(roc_auc_scores, columns=['n_hours', 'roc_auc_score'])
results_df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os

fig = plt.figure()
ax = sns.lineplot(x='n_hours', y='roc_auc_score', data=results_df, legend=True)
ax.set_title('Model performance in the holdout test dataset as a function of observation period')
ax.set_xlabel('Time after admission (hours)')
ax.set_ylabel('ROC AUC')
ax.set_ybound(0.8,0.92)

plt.tight_layout()
plt.show()

In [ ]:
# fig.savefig(os.path.join('/Users/jk1/Downloads', f'{outcome}_roc_auc_scores_over_time.png'), bbox_inches='tight')


## Performance at 24h

In [ ]:
from sklearn.utils import resample

roc_auc_scores_bs = []
ts = 24

n_iterations = 1000
for i in range(n_iterations):
    y_pred_bs, y_bs = resample(predictions_over_time[ts], test_y_np, replace=True)

    # evaluate model
    roc_auc_bs = roc_auc_score(y_bs, y_pred_bs)
    roc_auc_scores_bs.append(roc_auc_bs)

median_roc_auc = np.percentile(roc_auc_scores_bs, 50)
# get 95% interval
alpha = 100 - 95
lower_ci_roc_auc = np.percentile(roc_auc_scores_bs, alpha / 2)
upper_ci_roc_auc = np.percentile(roc_auc_scores_bs, 100 - alpha / 2)

print(median_roc_auc, lower_ci_roc_auc, upper_ci_roc_auc)